In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
paths = {
    'estratificacion': '../data/shp/ManzanaEstratificacion.shp',
    'uso_suelo': '../data/shp/ManzanaUsoSuelo_2019.shp',
    'viajes': '../data/csv/ViajesEODH_2019.csv',
    'zat': '../data/shp/ZAT.shp'
}

In [ ]:
df_viajes = pd.read_csv(paths['viajes'])
gdf_estratificacion = gpd.read_file(paths['estratificacion'])
gdf_uso_suelo = gpd.read_file(paths['uso_suelo'])
gdf_zat = gpd.read_file(paths['zat'], encoding='utf-8')

In [ ]:
# filtrar ZATs para Bogotá
gdf_zat = gdf_zat.query('NOMMun == "Bogotá"')

### 1. Crear matriz origen destino con conteos por ZAT

In [ ]:
df = pd.DataFrame(index=sorted(gdf_zat['ZAT'].unique()))

In [ ]:
df = df.join(df_viajes['zat_origen'].value_counts().rename('N_ORIGEN'))
df = df.join(df_viajes['zat_destino'].value_counts().rename('N_DESTINO'))

In [ ]:
# eliminar ZATs que no tienen viajes como origen o como destino
df = df.dropna()

### 2. Agregar estrato a las ZAT

In [ ]:
# crear interseccion entre ZATs y estratificacion
gdf_intersection = gpd.overlay(gdf_zat, gdf_estratificacion, how='intersection')

In [ ]:
# obtener estrato con mayor area total dentro de cada ZAT
agg = gdf_intersection.groupby(['ZAT','ESTRATO'])['geometry'].agg(lambda x: x.area.max())
top = agg.groupby(level=0, group_keys=False).nlargest(1)

In [ ]:
df = df.join(top.reset_index('ESTRATO').drop('geometry', axis=1))

In [ ]:
# eliminar ZATs que no tienen estrato (no hay interseccion)
df = df.dropna()

### 3. Agregar uso del suelo a las ZAT

In [ ]:
# crear interseccion entre ZATs y uso de suelo
gdf_intersection = gpd.overlay(gdf_zat, gdf_uso_suelo, how='intersection')

In [ ]:
# obtener uso con mayor area total dentro de cada ZAT
agg = gdf_intersection.groupby(['ZAT','USO'])['geometry'].agg(lambda x: x.area.max())
top = agg.groupby(level=0, group_keys=False).nlargest(1)

In [ ]:
df = df.join(top.reset_index('USO').drop('geometry', axis=1))

In [ ]:
# eliminar ZATs que no tienen uso (no hay interseccion)
df = df.dropna()